In [1]:
from termcolor import colored
import ipywidgets as widgets

%run COVID_Info_functions.ipynb
%run Domographics_functions.ipynb

In [2]:
def Get_Data(State_Name, ByPass_Update):
    State, State_Code = Get_State_Code(State_Name)
    
    print(colored('Obtaining Counties Geographical Data for the State of ' + State_Name, 'red', attrs=['bold']))
    Counties_Geo_df = Counties_Info(State_Code)
    print(colored('Obtaining Congressional Districts Geographical Data for the State of ' + State_Name, 
                  'red', attrs=['bold']))
    Districts_Geo_df = Congressional_Districts_Info(State_Code)
    
    print(colored('Obtaining Counties Demographics Data for the State of ' + State_Name, 'red', attrs=['bold']))
    Counties_Demo_df = Get_Counties_Demographics(State_Code)
    print(colored('Obtaining Congressional Districts Demographics Data for the State of ' + State_Name, 
                  'red', attrs=['bold']))
    Districts_Demo_df = Get_Districts_Demographics(State, State_Code)

    print(colored('Obtaining Counties COVID Data for the State of ' + State_Name, 'red', attrs=['bold']))
    if State == 'VA':
        Counties_COVID_df = Get_Cleaned_VA_COVID_data(ByPass_Update)
    else:
        print(colored('COVID data is only available for the state of Virginia right now', 
                      'blue', 'on_yellow', attrs=['bold']))
        return State_Code, [Counties_Geo_df, Districts_Geo_df, Counties_Demo_df, Districts_Demo_df]
    
    print(colored('Computing Congressional Districts COVID Data for the State of ' + State_Name, 
                  'red', attrs=['bold']))
    Districts_COVID_df = Get_CongressionalDistricts_COVID_data(State_Code, Counties_COVID_df)
    report_date_list = Districts_COVID_df['Report Date'].unique()
    
    Counties_COVID_df = Add_Pop_data(State_Code, 'County', Counties_COVID_df, 
                                     Counties_COVID_df.loc[:, 'Total Cases':'New Monthly Deaths'].columns.to_list())
    Districts_COVID_df = Add_Pop_data(State_Code, 'Congressional District', Districts_COVID_df, 
                                      Districts_COVID_df.loc[:, 'Total Cases':'New Monthly Deaths'].columns.to_list())
    
    list_of_dfs = [Counties_Geo_df, Districts_Geo_df, Counties_Demo_df, Districts_Demo_df, 
                   Counties_COVID_df, Districts_COVID_df, report_date_list]
    
    return State_Code, list_of_dfs

In [3]:
import time
def Get_Data_Map(State_Code, list_of_dfs):
    print(colored('Select options and click the Run button', 'red', 'on_yellow', attrs=['bold']))
    
    List_Keys_Dict = {('County', 'GEO'): 0, ('Congressional District', 'GEO'): 1, 
                      ('County', 'Partisan Voting Index'): 2, ('Congressional District', 'Partisan Voting Index'): 3,
                      ('County', 'Demographics, Age'): 2, ('Congressional District', 'Demographics, Age'): 3, 
                      ('County', 'Demographics, Race'): 2, ('Congressional District', 'Demographics, Race'): 3, 
                      ('County', 'Demographics, Income'): 2, ('Congressional District', 'Demographics, Income'): 3, 
                      ('County', 'COVID'): 4, ('Congressional District', 'COVID'): 5, 
                      ('None', 'None'): 7, ('County', 'None'): 7, ('Congressional District', 'None'): 7}
    
    if len(list_of_dfs) == 7:
        report_date_list = list_of_dfs[6]
    else:
        report_date_list = ['None']
        
    data_options = {'GEO': ['Population', 'Population Density', 'Land Area', 'Water Area'],
                    'COVID': ['Total Cases', 'Hospitalizations', 'Deaths', 
                              'New Daily Cases', 'New Daily Hospitalizations', 'New Daily Deaths', 
                              'New Weekly Cases', 'New Weekly Hospitalizations', 'New Weekly Deaths', 
                              'New Biweekly Cases', 'New Biweekly Hospitalizations', 'New Biweekly Deaths', 
                              'New Monthly Cases', 'New Monthly Hospitalizations', 'New Monthly Deaths', 
                              'Total Cases per Pop', 'Hospitalizations per Pop', 'Deaths per Pop', 
                              'New Daily Cases per Pop', 'New Daily Hospitalizations per Pop', 'New Daily Deaths per Pop', 
                              'New Weekly Cases per Pop', 'New Weekly Deaths per Pop', 
                              'New Weekly Hospitalizations per Pop', 'New Monthly Hospitalizations per Pop',
                              'New Monthly Cases per Pop', 'New Monthly Deaths per Pop',
                              'Hospitalizations Ratio', 'Deaths Ratio', 'Deaths per Hospitalizations'], 
                    'Partisan Voting Index': ['PVI_R', 'PVI_D'], 
                    'Demographics, Age': ['Total Population', 'Under 5 years', '5 to 14 years', '15 to 24 years', 
                                          'Under 18 years', '25 to 34 years', 'Under 35 years', '35 to 44 years', 
                                          '25 to 44 years', 'Under 45 years', '45 to 54 years', '55 to 64 years', 
                                          '45 to 64 years', '65 to 74 years',  '75 to 84 years', '65 years and over', 
                                          '85 years and over', 
                                          'Percent Under 5 years', 'Percent 5 to 14 years', 'Percent 15 to 24 years', 
                                          'Percent Under 18 years', 'Percent 25 to 34 years', 'Percent Under 35 years', 
                                          'Percent 35 to 44 years', 'Percent 25 to 44 years', 'Percent Under 45 years', 
                                          'Percent 45 to 54 years', 'Percent 55 to 64 years', 'Percent 45 to 64 years', 
                                          'Percent 65 to 74 years',  'Percent 75 to 84 years', 'Percent 65 years and over', 
                                          'Percent 85 years and over'], 
                    'Demographics, Race': ['White, Not Hispanic', 'Asian, Not Hispanic', 
                                           'Black or African American, Not Hispanic', 
                                           'American Indian & Alaska Native, Not Hispanic',
                                           'Native Hawaiin & Other Pacific Islander, Not Hispanic', 
                                           'Hispanic or Latino', 'Other, Not Hispanic', 'Two or More Races, Not Hispanic', 
                                           'Percent White, Not Hispanic', 'Percent Asian, Not Hispanic', 
                                           'Percent Black or African American, Not Hispanic', 
                                           'Percent American Indian & Alaska Native, Not Hispanic', 
                                           'Percent Native Hawaiin & Other Pacific Islander, Not Hispanic', 
                                           'Percent Hispanic or Latino', 'Percent Other, Not Hispanic', 
                                           'Percent Two or More Races, Not Hispanic'], 
                    'Demographics, Income': ['Median Income', 'Median Income, Male', 'Median Income, Female', 
                                             'Per Capita Income', 'Median Household Income', 'Median Income Female/Male']}
    
    date_options = {'GEO':['CD116'], 
                    'COVID':report_date_list[::-1], 
                    'Partisan Voting Index': ['2008, 2012 elections'],
                    'Demographics, Age':['2019'], 
                    'Demographics, Race':['2019'], 
                    'Demographics, Income':['2019']}

    Log_Scale_W = widgets.Checkbox(value = False, description = 'Log Scale', indent = False)

    level_W = widgets.RadioButtons(options = ['County', 'Congressional District'], 
                                   description = 'Level', 
                                   disabled = False)

    data_type_W = widgets.RadioButtons(options = data_options.keys(), 
                                       description = 'Data Type',  
                                       disabled=False)
    
    data_W = widgets.Dropdown(options = data_options[data_type_W.value])
    date_W = widgets.Dropdown(options = date_options[data_type_W.value])
    
    def update_data(*args):
        data_W.options = data_options[data_type_W.value]
        
    def update_date(*args):
        date_W.options = date_options[data_type_W.value]
        
    data_type_W.observe(update_data, 'value')
    data_type_W.observe(update_date, 'value')
    
    ##############################################################################################################
    def Map_State_Values(level, data_type, Log_Scale, date, data):
        k = List_Keys_Dict[(level, data_type)]
        
        if k >= len(list_of_dfs):
            return
        elif (k == 2) and (data_type == 'Partisan Voting Index'):
            print(colored('PVIs data for counties are not available', 'grey', 'on_yellow', attrs=['bold']))
            return
        
        if Log_Scale and (data_type == 'Partisan Voting Index'):
            Log_Scale = False
            print(colored('Log_Scale reset to False. PVI data have negative values!', 'grey', 'on_yellow', attrs=['bold']))
        
        df = list_of_dfs[k]
        
        if k == 4 or k == 5:
            mask = df['Report Date'] == date
            df = df[mask]
            if k == 4:
                df.set_index('County Code', inplace = True)
            elif k == 5:
                df.set_index('CDistrict', inplace = True)
        
        return Map_State(State_Code, level, df, data, Log_Scale)
    ###############################################################################################################
    
    widgets.AppLayout(left_sidebar = level_W, right_sidebar = data_type_W, footer = data_W)
    widget = widgets.interactive(Map_State_Values, {'manual' : True, 'manual_name' : 'Run'}, 
                                 level = level_W, data_type = data_type_W, Log_Scale = Log_Scale_W, 
                                 date = date_W, data = data_W)
    
    controls = widgets.HBox(widget.children[:-1], layout = widgets.Layout(flex_flow='row wrap'))
    output = widget.children[-1]
    display(widgets.VBox([controls, output]))